In [1]:
import requests
from urllib import parse
import time
from datetime import datetime
import cx_Oracle as db
import random
import math

import os

In [2]:
def binarySearchAsc(mItem, mList):
    
    startIndex = 0
    endIndex = len(mList) - 1
    
    while(startIndex <= endIndex):
        target = int((startIndex + endIndex) / 2)
        
        if(mItem == mList[target]):
            return [True, target]
        elif(mItem < mList[target]):
            endIndex = target - 1
        else:
            startIndex = target + 1
            
    if(endIndex == -1):
        return [False, startIndex]
    if(startIndex > endIndex):
        return [False, startIndex]
    return [False, target]

def binarySearchtuple(mItem, mList, key):

    startIndex = 0
    endIndex = len(mList) - 1
    
    while(startIndex <= endIndex):
        target = int((startIndex + endIndex) / 2)
        
        if(mItem == mList[target][key]):
            return [True, target]
        elif(mItem < mList[target][key]):
            endIndex = target - 1
        else:
            startIndex = target + 1
            
    if(endIndex == -1):
        return [False, startIndex]
    if(startIndex > endIndex):
        return [False, startIndex]
    return [False, target]

In [6]:
#테이블 3900 최대 117000 30
tablesize = 3900

conn = db.connect('lolpredict', 'korhrd', '127.0.0.1:1521')
cursor = conn.cursor()

sql = 'select * from userinfo where isComplete = 0'
cursor.execute(sql)
userList = cursor.fetchall()

sql = 'select matchid from matchinfo order by matchid'
cursor.execute(sql)
matchList = cursor.fetchall()

requestHeader = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0",
    "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-8f1b0671-b2df-4b39-9c32-d7a738fb20b7"
}

maxlength = 0
while(len(userList)):
    i0 = random.randint(0, len(userList) - 1)
    
    isnotFound = False
    nameUrl = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + userList[i0][0]
    nameCode = 0
    while(nameCode != 200):
        puuid = requests.get(nameUrl, headers=requestHeader)
        nameCode = puuid.status_code
        time.sleep(2)
        print(puuid.status_code)
            
        if(nameCode == 404):
            isnotFound = True
            break
    
    if(isnotFound == True):
        sql = "update userinfo SET iscomplete = 1 where username = '{}'".format(userList[i0][0])
        cursor.execute(sql)
        conn.commit()
        continue
    
    puuidUrl = "https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/" + puuid.json()['puuid'] + "/ids?start=0&count=100"
    puuidCode = 0
    while(puuidCode != 200):
        puuidResult = requests.get(puuidUrl, headers=requestHeader)
        puuidCode = puuidResult.status_code
        time.sleep(2)
        print(puuidResult.status_code)
        
        if(nameCode == 404):
            isnotFound = True
            break
    
    if(isnotFound == True):
        sql = "update userinfo SET iscomplete = 1 where username = '{}'".format(userList[i0][0])
        cursor.execute(sql)
        conn.commit()
        continue
        
    for matchid in puuidResult.json():
        searchMatch = binarySearchtuple(matchid, matchList, 0)
        if searchMatch[0] == True:
            continue
        
        matchUrl = "https://asia.api.riotgames.com/lol/match/v5/matches/" + matchid
        matchCode = 0
        while(matchCode != 200):
            matchResult = requests.get(matchUrl, headers=requestHeader)
            matchCode = matchResult.status_code
            time.sleep(2)
            print(matchResult.status_code)
            
        for user in matchResult.json()['info']['participants']:
            searchUser = binarySearchtuple(user['summonerName'], userList, 0)
            if searchUser[0] == True:
                continue
                
            sql = "insert into userInfo values('{}', {})".format(user['summonerName'], 0)
            
            try:
                cursor.execute(sql)
                conn.commit()
            except:
                continue
        
        longjson = str(matchResult.json())
        longjson = longjson.replace("'", "\\\"")
        splitjson = []
        for i in range(math.ceil(len(longjson) / tablesize)):
            splitjson.append(longjson[i * tablesize : (i + 1) * tablesize])
        
        sql = "insert into matchInfo(matchid"
        for i in range(len(splitjson)):
            sql += ", matchjson{}".format(i)
        sql += ") values('{}'".format(matchid)
        for i in range(len(splitjson)):
            sql += ", '{}'".format(splitjson[i])
        sql += ')'
            
        try:
            cursor.execute(sql)
            conn.commit()
        except:
            continue
        
    sql = "update userinfo SET iscomplete = 1 where username = '{}'".format(userList[i0][0])
    cursor.execute(sql)
    conn.commit()
    
    sql = 'select * from userinfo where isComplete = 0 order by username'
    cursor.execute(sql)
    userList = cursor.fetchall()
    
    sql = 'select matchid from matchinfo order by matchid'
    cursor.execute(sql)
    matchList = cursor.fetchall()

200
200
200
200
200
200
200
200
200
200
200
200
200


KeyboardInterrupt: 

In [178]:
sql = "insert into matchInfo(matchid"
for i in range(len(splitjson)):
    sql += ", matchjson{}".format(i)
sql += ") values('{}'".format(matchid)
for i in range(len(splitjson)):
    sql += ", '{}'".format(splitjson[i])
sql += ')'

sql

"insert into matchInfo(matchid, matchjson0, matchjson1, matchjson2, matchjson3, matchjson4, matchjson5, matchjson6, matchjson7, matchjson8, matchjson9, matchjson10, matchjson11, matchjson12, matchjson13, matchjson14, matchjson15, matchjson16, matchjson17, matchjson18, matchjson19, matchjson20, matchjson21, matchjson22, matchjson23, matchjson24, matchjson25, matchjson26) values('KR_6007289435', '{'metadata': {'dataVersion': '2', 'matchId': 'KR_6007289435', 'participants': ['1tqVSD5vTH8qdtR6HfZK2wfHGI4eYTxXY2-C0pfkL4eyuw0sv7lZoj_SMtIOERwh4kVSinOvQ82p0Q', 'Y0M4NXfd6-72tsshmkZI4uRRs4hVmBNkHs5exrqOJXDyTyvkcIJoyoCX9R4DOqc-Mt2KD8NuWQu7dw', 'jB6NmGAJRBMuchDPoZCtLZrQNLeiOa5GVUS5ThIEkZcBdfS-iaVt1hpEvDrNARLxQ0Ml1YIwUtnNGQ', 'MIIaSoSQ2GP-mjOLkt_GFU5VFQcvmjUTOFUTkxaeO1JrvnVYvSZHcLP6h02c0fMIPTn6uzjFs2-k8w', 'xusQMKCPXIcjgDlV5AXu4mww4Fmpk-XWbtaoK2NnXX5Q69pHNHegK_XqUUSGSnDYCcnErmo3u0b-lA', 'Pgs2J7o42qgvSlsEXXeZAaKY6_Lat3TRbA9VAiKja6E84QGUc8YAFxmBcijQGeoLXDkj-L7MSUwTrg', 'UBb2kF98V0m3_FmCn3JoZW0YM-VvrR

In [116]:
splitjson

["{'metadata': {'dataVersion': '2', 'matchId': 'KR_6007289435', 'participants': ['1tqVSD5vTH8qdtR6HfZK2wfHGI4eYTxXY2-C0pfkL4eyuw0sv7lZoj_SMtIOERwh4kVSinOvQ82p0Q', 'Y0M4NXfd6-72tsshmkZI4uRRs4hVmBNkHs5exrqOJXDyTyvkcIJoyoCX9R4DOqc-Mt2KD8NuWQu7dw', 'jB6NmGAJRBMuchDPoZCtLZrQNLeiOa5GVUS5ThIEkZcBdfS-iaVt1hpEvDrNARLxQ0Ml1YIwUtnNGQ', 'MIIaSoSQ2GP-mjOLkt_GFU5VFQcvmjUTOFUTkxaeO1JrvnVYvSZHcLP6h02c0fMIPTn6uzjFs2-k8w', 'xusQMKCPXIcjgDlV5AXu4mww4Fmpk-XWbtaoK2NnXX5Q69pHNHegK_XqUUSGSnDYCcnErmo3u0b-lA', 'Pgs2J7o42qgvSlsEXXeZAaKY6_Lat3TRbA9VAiKja6E84QGUc8YAFxmBcijQGeoLXDkj-L7MSUwTrg', 'UBb2kF98V0m3_FmCn3JoZW0YM-VvrRCaLfL8REhF_SBICiwWdIj2UmL5iRP8EbeauJ3NeNF0Pes0OQ', 'S1dGGpmQTmsbU1zEg8Eo5DnRfXU_Y-qCEEwFuy6rQBBdUZ3FgcGR0b6dQQoT6873Nah8Pmksqmq3Nw', '0foEF2WbabHcUL0trAjh_MMrPk5Ec-cRfs_-TOJPKFHxiOIovmKgw1xOEIu1Q7JS4EyAgcq1cH29sA', 'ZuAaT2UFNnJ-pDjqlOhJSV7C7YILDj_ZMDjkfTjnv5qUI-613bCzN0gqekVPVoBmJBBzmzXKE53YyA']}, 'info': {'gameCreation': 1657223845000, 'gameDuration': 1189, 'gameEndTimestamp': 1657225048865, 